# Dinh et al. (2019)

## A data-driven approach to predicting diabetes and cardiovascular disease with machine learning

URL: https://bmcmedinformdecismak.biomedcentral.com/articles/10.1186/s12911-019-0918-5


## Brief Summary

Dinh et al. (2019) uses different ML models (logistic regression, support vector machines, random forest, and gradient boosting) on NHANES dataset to predict i) Diabetes and ii) Cardiovascular disease ("CVD").

**Goal**: Identification mechanism for patients at risk of diabetes and cardiovascular diseases and key contributors to diabetes .

**Results**:

Best scores:

- CVB prediction based on 131 NHANES variables achieved an AU-ROC score of 83.9% .
- Diabetes prediction based on 123 NHANES variables achieved an AU-ROC score of 95.7% .
- Pre-diabetic prediction based on 123 NHANES variables achieved an AU-ROC score of 84.4% .
- Top 5 predictors in diabetes patients were 1) `waist size`, 2) `age`, 3) `self-reported weight`, 4) `leg length`, 5) `sodium intake`.



This notebook replicates the results of the paper. The structure follows the following steps: 

1. NHANES data 
2. Pre-processing of the data
3. Transformation of the data
4. Train/Test Split 
5. CV 10-fold
6. Training monitoring using MLflow
7. Get metric results (AUC)


The structure of the analysis emulates the Figure 1 from the paper: 

![Fig 1 from Dinh et al. 2019](https://raw.githubusercontent.com/pipegalera/ml_diabetes/main/images/dinh_2019_Fig1.png)


In [4]:
import pandas as pd
import numpy as np

In [5]:
SEED = 4208
DATA_PATH = "/root/dev/ml_diabetes/data/raw_data/NHANES/"
DINH_DOCS_PATH = "/root/dev/ml_diabetes/data/processed/NHANES/"

## 1. HNANES data

### Covariates and Targets 

- Source: https://www.cdc.gov/nchs/index.htm
- Downloaded raw data via: `notebooksnhanes_data_backfill`


The paper did not mention what variables they use from NHANES. I emailed the author in the correspondence section of the paper to try to get the list of variables they used, but no answer from him yet.

Please notice that NHANES have more than 3900 variables, therefore without the list of the specific variables used it is impossible to fully replicate the paper.

For now, I will consider the variables taken from [Figure 5](https://raw.githubusercontent.com/pipegalera/ml_diabetes/main/images/dinh_2019_Fig5.png) and [Figure 6](https://raw.githubusercontent.com/pipegalera/ml_diabetes/main/images/dinh_2019_Fig6.png) of the paper. I compiled them by hand in an Excel file using NHANES search tool for variables (see: `processed/NHANES/dinh_2019_variables_doc.xlsx`).


- `Case I: Diabetes`

    - Glucose >= 126 mg/dL. OR;
    - 1 to the question "Have you ever been told by a doctor that you have diabetes?"

- `Case II: Undiagnosed Diabetes`

    - Glucose >= 126 mg/dL. AND;
    - "No" to the question "Have you ever been told by a doctor that you have diabetes?"

- `Cardiovascular disease`

    - 1 to any of the the questions "Have you ever been told by a doctor that you had congestive heart failure, coronary heart disease, a heart attack, or a stroke?"

- `Pre diabetes`

    - Glucose 125 >= 100 mg/dL

In [6]:
dinh_2019_vars = pd.read_excel(f"{DINH_DOCS_PATH}dinh_2019_variables_doc.xlsx")

dinh_2019_vars[["Variable Name", "NHANES Name"]].head(15)

,Variable Name,NHANES Name
0,Age,RIDAGEYR
1,Alcohol consumption,ALQ130
2,Alcohol intake,DRXTALCO
3,"Alcohol intake, First Day",DR1TALCO
4,"Alcohol intake, Second Day",DR2TALCO
5,Arm circumference,BMXARMC
6,Arm length,BMXARML
7,Blood osmolality,LBXSOSSI
8,Blood relatives have diabetes,MCQ250A
9,Blood urea nitrogen,LBDSBUSI


For the complete list of variables, check the file `dinh_2019_variables_doc.xlsx` under NHANES data folder.

NHANES data is made by multiple files (see `NHANES` unde data folder) that have to be compiled together. The data was downloaded automatically via script, all the files converted from SAS to parquet, and the files were stacked and merged based on the individual index ("SEQN"). For more details please check the `nhanes_data_backfill` notebook. 

Plese notice that no transformation are made to the covariates, the files were only arranged and stacked together. 

In [7]:
df = pd.read_parquet(f"{DATA_PATH}dinh_raw_data.parquet")


In [8]:
df.head()

,SEQN,YEAR,RIDAGEYR,ALQ130,DRXTALCO,DR1TALCO,DR2TALCO,BMXARMC,BMXARML,LBXSOSSI,...,SEQ060,DIQ010,MCQ160B,MCQ160b,MCQ160C,MCQ160c,MCQ160E,MCQ160e,MCQ160F,MCQ160f
0,1.0,1999-2000,2.0,NaN,0.00,NaN,NaN,15.2,18.6,NaN,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,1999-2000,77.0,1.0,0.00,NaN,NaN,29.8,38.2,288.0,...,NaN,2.0,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN
2,3.0,1999-2000,10.0,NaN,0.00,NaN,NaN,19.7,25.5,NaN,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,1999-2000,1.0,NaN,0.00,NaN,NaN,16.4,20.4,NaN,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,1999-2000,49.0,3.0,34.56,NaN,NaN,35.8,39.7,276.0,...,NaN,2.0,2.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN


In [9]:
df.shape[0]

82091

In [10]:
df.columns

Index(['SEQN', 'YEAR', 'RIDAGEYR', 'ALQ130', 'DRXTALCO', 'DR1TALCO',
       'DR2TALCO', 'BMXARMC', 'BMXARML', 'LBXSOSSI', 'MCQ250A', 'LBDSBUSI',
       'BMXBMI', 'DRXTCAFF', 'DR1TCAFF', 'DR2TCAFF', 'DR1TCALC', 'DR2TCALC',
       'DRXTCALC', 'DR1TCARB', 'DR2TCARB', 'DRXTCARB', 'LBXSNASI', 'LBXSCLSI',
       'MCQ300c', 'MCQ300C', 'BPXDI1', 'BPXDI4', 'BPXDI2', 'BPXDI3',
       'RIDRETH1', 'DR1TFIBE', 'DR2TFIBE', 'DRXTFIBE', 'LBXSGTSI', 'HSD010',
       'HUQ010', 'LBDHDLSI', 'LBDHDDSI', 'BMXHT', 'BPQ080', 'INDHHIN2',
       'DRXTKCAL', 'DR1TKCAL', 'DR2TKCAL', 'LBDLDLSI', 'BMXLEG', 'LBDLYMNO',
       'LBXMCVSI', 'BPXPLS', 'WHD140', 'DR1TSODI', 'DR2TSODI', 'DRDTSODI',
       'BPXSY1', 'BPXSY4', 'BPXSY2', 'BPXSY3', 'LBDTCSI', 'LBDSTRSI',
       'BMXWAIST', 'BMXWT', 'LBXWBCSI', 'LBXSASSI', 'LBXGLUSI', 'LBDGLUSI',
       'RHQ141', 'RHD143', 'SEQ060', 'DIQ010', 'MCQ160B', 'MCQ160b', 'MCQ160C',
       'MCQ160c', 'MCQ160E', 'MCQ160e', 'MCQ160F', 'MCQ160f'],
      dtype='object')

## 2. Pre-processing and Data modeling


### 2.1 Extreme values and replacing Missing/Don't know answers

> The preprocessing stage also converted any undecipherable values (errors in datatypes and standard formatting) from the database to null representations.

For this, I've checked the variables according to their possible values in the NHANES documentation (https://wwwn.cdc.gov/nchs/nhanes/search/default.aspx). I did not found any any extreme value out of the possible ranges. However, the data is reviwed and updated after the survey, so it might be that the NCHS applied some fixes after they saw them. 


I have replaced "Don't know" and "Refused" for NA values and converted the intial encoding of the categorical variables to the real values in the survey - given that the encoding is not consistent accross years. For the model, I will encode the variables myself so I don't have to jungle NHANES encoding. 

All the variables can by found at  https://wwwn.cdc.gov/nchs/nhanes/search/default.aspx

In [11]:
# These are numerical variables that have coded "Don't know" and "Refused" as numerical
df['ALQ130'] = df['ALQ130'].replace([77, 99, 777, 999], np.nan)
df['WHD140'] = df['WHD140'].replace([7777, 77777, 9999, 99999], np.nan)


### 2.2 Homogenize variables that are the same but are called diffrent in different NHANES years

Intake variables went from 1 day in 1999 to 2001 to 2 days from 2003 on, therefore the variable has to be homogenized. Dinh et al. (2019) do not specify which examination records the authors, but my best guess is that they problably took the average of both days that the examination was performed. 

This situation happends with:

- Alcohol intake (`DRXTALCO`, `DR1TALCO`, `DR2TALCO`)
- Caffeine intake (`DRXTCAFF`, `DR1TCAFF`, `DR2TCAFF`)
- Calcium intake (`DRXTCALC`, `DR1TCALC`, `DR2TCALC`)
- Carbohydrate intake (`DRXTCARB`, `DR1TCARB`, `DR2TCARB`)
- Fiber intake (`DRXTFIBE`, `DR1TFIBE`, `DR2TFIBE`)
- Kcal intake (`DRXTKCAL`, `DR1TKCAL`, `DR2TKCAL`)
- Sodium intake (`DRDTSODI`, `DR1TSODI`, `DR2TSODI`)


Also, small changes in same quesion format are registered with different codes. Examples: 

- `MCQ250A`, `MCQ300C` and `MCQ300c`
- `LBDHDDSI` and `LBDHDLSI`
- `LBXGLUSI` and `LBDGLUSI`
- `SEQ060`,`RHQ141`, and `RHD143`

And same questions are coded differnetly as well:

- `MCQ160B` and `MCQ160b`
- `MCQ160C` and `MCQ160c`
- `MCQ160E` and `MCQ160e`
- `MCQ160F` and `MCQ160F`


It can be seen here:

In [12]:
# Similar questions (or the same) with different NHANES variable codes
var_docs = pd.read_excel(f"{DINH_DOCS_PATH}dinh_2019_variables_doc.xlsx")

filtered_var_docs = var_docs[var_docs['NHANES Name'].isin(
    ['MCQ250A', 'MCQ300C', 'MCQ300c', 
     'LBDHDDSI', 'LBDHDLSI', 'LBXGLUSI', 'LBDGLUSI']
)]

To fix that, I will create a function that creates an average of the Intake variable of Day 1 and Day and average them, givin only one variable - for example "Alcohol_Intake" instead of having 'DRXTALCO', 'DR1TALCO', 'DR2TALCO'.

In [13]:
def create_intake_new_column(df, day0_col, day1_col, day2_col):
    return np.where(df[day0_col].isna(),
                    df[[day1_col, day2_col]].mean(axis=1, skipna=True),
                    df[day0_col])

# Assuming df is already defined and loaded with data

# Create new columns
df['Alcohol_Intake'] = create_intake_new_column(df, 'DRXTALCO', 'DR1TALCO', 'DR2TALCO')
df['Caffeine_Intake'] = create_intake_new_column(df, 'DRXTCAFF', 'DR1TCAFF', 'DR2TCAFF')
df['Calcium_Intake'] = create_intake_new_column(df, 'DRXTCALC', 'DR1TCALC', 'DR2TCALC')
df['Carbohydrate_Intake'] = create_intake_new_column(df, 'DRXTCARB', 'DR1TCARB', 'DR2TCARB')
df['Fiber_Intake'] = create_intake_new_column(df, 'DRXTFIBE', 'DR1TFIBE', 'DR2TFIBE')
df['Kcal_Intake'] = create_intake_new_column(df, 'DRXTKCAL', 'DR1TKCAL', 'DR2TKCAL')
df['Sodium_Intake'] = create_intake_new_column(df, 'DRDTSODI', 'DR1TSODI', 'DR2TSODI')

# Coalesce functions
df['Relative_Had_Diabetes'] = df['MCQ250A'].combine_first(df['MCQ300C']).combine_first(df['MCQ300c'])
df['Told_CHF'] = df['MCQ160B'].combine_first(df['MCQ160b'])
df['Told_CHD'] = df['MCQ160C'].combine_first(df['MCQ160c'])
df['Told_HA'] = df['MCQ160E'].combine_first(df['MCQ160e'])
df['Told_stroke'] = df['MCQ160F'].combine_first(df['MCQ160f'])
df['Pregnant'] = df['SEQ060'].combine_first(df['RHQ141']).combine_first(df['RHD143'])
df['HDL_Cholesterol'] = df['LBDHDLSI'].combine_first(df['LBDHDDSI'])
df['Glucose'] = df['LBXGLUSI'].combine_first(df['LBDGLUSI'])

# Delete old columns that are not needed
columns_to_drop = ['DRXTALCO', 'DR1TALCO', 'DR2TALCO', 'DRXTCAFF', 'DR1TCAFF', 'DR2TCAFF',
                   'DRXTCALC', 'DR1TCALC', 'DR2TCALC', 'DRXTCARB', 'DR1TCARB', 'DR2TCARB',
                   'DRXTFIBE', 'DR1TFIBE', 'DR2TFIBE', 'DRXTKCAL', 'DR1TKCAL', 'DR2TKCAL',
                   'DRDTSODI', 'DR1TSODI', 'DR2TSODI', 'MCQ250A', 'MCQ300C', 'MCQ300c',
                   'MCQ160B', 'MCQ160b', 'MCQ160C', 'MCQ160c', 'MCQ160E', 'MCQ160e',
                   'MCQ160F', 'MCQ160f', 'SEQ060', 'RHQ141', 'RHD143',
                   'LBDHDLSI', 'LBDHDDSI', 'LBXGLUSI', 'LBDGLUSI']

df = df.drop(columns=columns_to_drop)

In [14]:
df[df['Relative_Had_Diabetes'].isna()]['YEAR'].unique()

array(['1999-2000', '2001-2002', '2003-2004', '2005-2006', '2007-2008',
       '2009-2010', '2011-2012', '2013-2014'], dtype=object)

### 2.3 Choosing between different readings in Blood analysis 

[From NHANES](https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/BPX_H.htm): 

> After resting quietly in a seated position for 5 minutes and once the participants maximum inflation level (MIL) has been determined, three consecutive blood pressure readings are obtained. If a blood pressure measurement is interrupted or incomplete, a fourth attempt may be made. All BP determinations (systolic and diastolic) are taken in the mobile examination center (MEC). 

In Dinh et al. (2019) the authors do not say which readings are taking, but I'm assuming they take the last one to avoid the [white coat syndrom](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5352963/) and for data consistency.

In [15]:
# Create new columns
df['Diastolic_Blood_Pressure'] = df['BPXDI4'].combine_first(df['BPXDI3']).combine_first(df['BPXDI2']).combine_first(df['BPXDI1'])
df['Systolic_Blood_Pressure'] = df['BPXSY4'].combine_first(df['BPXSY3']).combine_first(df['BPXSY2']).combine_first(df['BPXSY1'])

# Delete old columns that are not needed
columns_to_drop = ['BPXDI4', 'BPXDI3', 'BPXDI2', 'BPXDI1',
                   'BPXSY4', 'BPXSY3', 'BPXSY2', 'BPXSY1']

df = df.drop(columns=columns_to_drop)

### 2.4 Discretional trimming of the data according to the authors

> In our study, all datasets were limited to non-pregnant subjects and adults of at least twenty years of age.

In [16]:

# Filter out pregnant individuals and those under 20 years old
cond_1 = (df['Pregnant'].isna()) | (df['Pregnant'] != 1)
cond_2 =(df['RIDAGEYR'] >= 20)
df = df[cond_1 & cond_2]

#df = df.drop(columns=['Pregnant'])

In [17]:
df.shape[0]

42655

### 2.5 Creating the  Target Variables

Tables 1 & 3 from Dinh et al. 2019:

From [Tables 1 & 3 from Dinh et al. 2019](https://raw.githubusercontent.com/pipegalera/ml_diabetes/main/images/dinh_2019_Table1_3.png):

`Diabetes = 1` if

- Glucose >= 126 mg/dL. OR;
- "Yes" to the question "Have you ever been told by a doctor that you have diabetes?"

`undiagnosed diabetes = 1` if

- Glucose >= 126 mg/dL. AND;
- "No" to the question "Have you ever been told by a doctor that you have diabetes?" and had a blood glucose level greater than or equal

`pre diabetes = 1` if

- Glucose 125 >= 100 mg/dL

`CVD = 1` if

- "Yes" to any of the the questions "Have you ever been told by a doctor that you had congestive heart failure, coronary heart disease, a heart attack, or a stroke?"



In [18]:
df['DIQ010'].unique() #1=yes, 2=no

array([ 2.,  1.,  3.,  9., nan,  7.])

In [19]:
df['Diabetes_Case_I'] = np.where(
  (df['Glucose'] > 7.0) | (df['DIQ010'] == 1), 
  1, 
  0)


df['Diabetes_Case_II'] = np.where(
  (df['Diabetes_Case_I'] == 0) & (df['Glucose'] >= 5.6) & (df['Glucose'] < 7.0), 
  1, 
  0)

df['CVD'] = np.where(
    (df['Told_CHF'] == 1) | (df['Told_CHD'] == 1) | (df['Told_HA'] == 1) | (df['Told_stroke'] == 1),
    1, 
    0)

# Drop the unnecessary columns
#df = df.drop(columns=['Told_CHF', 'Told_CHD', 'Told_HA', 'Told_stroke', 'Glucose', 'DIQ010'])


### 2.6 Column name formatting

In [20]:
df = df.rename(columns={
    'ALQ130': 'Alcohol_consumption',
    'BMXARMC': 'Arm_circumference',
    'BMXARML': 'Arm_length',
    'BMXBMI': 'Body_mass_index',
    'BMXHT': 'Height',
    'BMXLEG': 'Leg_length',
    'BMXWAIST': 'Waist_circumference',
    'BMXWT': 'Weight',
    'BPQ080': 'Told_High_Cholesterol',
    'BPXPLS': 'Pulse',
    'HSD010': 'General_health',
    'HUQ010': 'Health_status',
    'INDHHIN2': 'Household_income',
    'LBXSCLSI': 'Chloride',
    'LBXSNASI': 'Sodium',
    'LBDLDLSI': 'LDL_cholesterol',
    'LBDLYMNO': 'Lymphocytes',
    'LBDSBUSI': 'Blood_urea_nitrogen',
    'LBDSTRSI': 'Triglycerides',
    'LBDTCSI': 'Total_cholesterol',
    'LBXMCVSI': 'Mean_cell_volume',
    'LBXSASSI': 'Aspartate_aminotransferase_AST',
    'LBXSGTSI': 'Gamma_glutamyl_transferase',
    'LBXSOSSI': 'Osmolality',
    'LBXWBCSI': 'White_blood_cell_count',
    'RIDAGEYR': 'Age',
    'RIDRETH1': 'Race_ethnicity',
    'WHD140': 'Self_reported_greatest_weight',
    'YEAR': 'Survey_year'
})


In [21]:
df.columns

Index(['SEQN', 'Survey_year', 'Age', 'Alcohol_consumption',
       'Arm_circumference', 'Arm_length', 'Osmolality', 'Blood_urea_nitrogen',
       'Body_mass_index', 'Sodium', 'Chloride', 'Race_ethnicity',
       'Gamma_glutamyl_transferase', 'General_health', 'Health_status',
       'Height', 'Told_High_Cholesterol', 'Household_income',
       'LDL_cholesterol', 'Leg_length', 'Lymphocytes', 'Mean_cell_volume',
       'Pulse', 'Self_reported_greatest_weight', 'Total_cholesterol',
       'Triglycerides', 'Waist_circumference', 'Weight',
       'White_blood_cell_count', 'Aspartate_aminotransferase_AST', 'DIQ010',
       'Alcohol_Intake', 'Caffeine_Intake', 'Calcium_Intake',
       'Carbohydrate_Intake', 'Fiber_Intake', 'Kcal_Intake', 'Sodium_Intake',
       'Relative_Had_Diabetes', 'Told_CHF', 'Told_CHD', 'Told_HA',
       'Told_stroke', 'Pregnant', 'HDL_Cholesterol', 'Glucose',
       'Diastolic_Blood_Pressure', 'Systolic_Blood_Pressure',
       'Diabetes_Case_I', 'Diabetes_Case_II', 'CV

In [22]:
df

,SEQN,Survey_year,Age,Alcohol_consumption,Arm_circumference,Arm_length,Osmolality,Blood_urea_nitrogen,Body_mass_index,Sodium,...,Told_HA,Told_stroke,Pregnant,HDL_Cholesterol,Glucose,Diastolic_Blood_Pressure,Systolic_Blood_Pressure,Diabetes_Case_I,Diabetes_Case_II,CVD
1,2.0,1999-2000,77.0,1.0,29.8,38.2,288.0,6.80,24.90,144.1,...,2.0,2.0,NaN,1.39,4.646,56.0,98.0,0,0,0
4,5.0,1999-2000,49.0,3.0,35.8,39.7,276.0,5.70,29.10,137.5,...,2.0,2.0,NaN,1.08,5.550,82.0,122.0,0,0,0
6,7.0,1999-2000,59.0,NaN,31.7,38.1,283.0,3.60,29.39,143.2,...,2.0,2.0,2.0,2.73,4.756,82.0,124.0,0,0,0
9,10.0,1999-2000,43.0,1.0,37.6,43.0,281.0,4.60,30.94,140.9,...,2.0,2.0,NaN,1.31,4.989,96.0,142.0,0,0,0
11,12.0,1999-2000,37.0,3.0,37.2,40.0,283.0,7.10,30.62,141.3,...,2.0,2.0,NaN,0.98,4.606,100.0,176.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82082,83723.0,2013-2014,61.0,2.0,35.8,40.0,280.0,6.07,33.10,138.0,...,2.0,2.0,NaN,1.27,8.826,68.0,138.0,1,0,0
82083,83724.0,2013-2014,80.0,NaN,31.0,36.0,287.0,9.28,24.90,141.0,...,2.0,2.0,NaN,1.32,NaN,66.0,168.0,0,0,0
82085,83726.0,2013-2014,40.0,NaN,31.0,39.0,NaN,NaN,26.80,NaN,...,2.0,2.0,NaN,NaN,NaN,NaN,NaN,0,0,0
82086,83727.0,2013-2014,26.0,3.0,29.9,35.2,285.0,4.64,24.50,143.0,...,2.0,2.0,NaN,1.42,5.995,76.0,112.0,0,1,0


## 3. Model Development


### 3.1 Train/Test split

The paper do a 80/20 split to train the model, trying to keep the target class proportions of the NHANES population in train and test sets:

> Downsampling was used to produce a balanced 80/20 train/test split.

First, I created a split based on the interaction of all the targets to keep the balance of the target labels. Afterwards, I created a quick R function to check it:

In [31]:
from sklearn.model_selection import train_test_split

target = 'Diabetes_Case_I'

y = df[target]
X = df.drop(columns= ['Diabetes_Case_I', 
                      'Diabetes_Case_II', 
                      'CVD', 
                      'SEQN', 'Survey_year'])

strata = (df['Diabetes_Case_I'].astype(str) + 
          '_' + df['Diabetes_Case_II'].astype(str) 
          + '_' + df['CVD'].astype(str))

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2, 
                                                    random_state=SEED, 
                                                    stratify=strata)

del strata

In [45]:
def percent_target(data, target):
    percent_target = round(data.reset_index()[target].sum()/len(data),4)*100
    object_name = [name for name in globals() if globals()[name] is data][0]
    print(f"In the {object_name} dataset, percentage of cases of {target} is: {percent_target}%")

# Assuming 'df', 'train_data', and 'test_data' are already defined
percent_target(df, "Diabetes_Case_I")
percent_target(y_train, "Diabetes_Case_I")
percent_target(y_test, "Diabetes_Case_I")


In the df dataset, percentage of cases of Diabetes_Case_I is: 13.43%
In the y_train dataset, percentage of cases of Diabetes_Case_I is: 13.43%
In the y_test dataset, percentage of cases of Diabetes_Case_I is: 13.43%


### 3.2 ML pipeline

I will use `caret` random search cross-validation implementation, as it might be the closest library to `sklearn` in python. More [here](https://topepo.github.io/caret/random-hyperparameter-search.html).

The models are not very intensive in terms of hypertuning (with the exception of XGBoost that I will probably set a time limit). 

The information about the hypertunning of the model in the paper is rather confusing:


> For each model, a grid-search approach with parallelized performance evaluation for model parameter tuning was used to generate the best model parameters. Next, each of the models underwent a 10-fold cross-validation (10 folds of training and testing with randomized data-split)

It doesn't make so much sense to do grid-search on the entire training set before a cross-validation. The reason is that you would basically train the data before the validation set that you are using to check if the model generalize well. And then the whole purpose of the cross-validation is to check model fit outside of the training set - it would lose all purpose. 

My best guess is that they used `GridSearchCV/RandomSearchCV` from sklearn and they assumed that the grid search go first. 

In the pipeline I also included encoding and standarization similar to the one done by the authors: 

> Normalization was performed on the data using the following standardization model: x' = x−x^/σ 


The ML pipelines will follow the these steps, imitatting the paper:

1. Apply preprocessing (encoding and standarization) to the initial data. 
2. Split data into train (80%) and test (20%) sets.
3. Train and tune the 4 standard models using 10-fold CV random grid search on the train set.
4. Create the Weighted Ensemble Model (WEM) based on the 4 models and weighted by AUC scores.
5. Evaluate the final model on the test set.

In [69]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from scipy.stats import uniform, randint
from sklearn.base import BaseEstimator
from sklearn.metrics import roc_auc_score, recall_score, precision_score, f1_score


In [49]:
# Categorical variables
categorical_vars = [
    'Race_ethnicity',
    'General_health',
    'Health_status',
    'Told_High_Cholesterol',
    'Household_income',
    'Relative_Had_Diabetes'
]

# Numerical variables
numerical_vars = [
    'Age',
    'Alcohol_consumption',
    'Arm_circumference',
    'Arm_length',
    'Osmolality',
    'Blood_urea_nitrogen',
    'Body_mass_index',
    'Chloride',
    'Sodium',
    'Gamma_glutamyl_transferase',
    'Height',
    'LDL_cholesterol',
    'Leg_length',
    'Lymphocytes',
    'Mean_cell_volume',
    'Pulse',
    'Self_reported_greatest_weight',
    'Total_cholesterol',
    'Triglycerides',
    'Waist_circumference',
    'Weight',
    'White_blood_cell_count',
    'Aspartate_aminotransferase_AST',
    'Alcohol_Intake',
    'Caffeine_Intake',
    'Calcium_Intake',
    'Carbohydrate_Intake',
    'Fiber_Intake',
    'Kcal_Intake',
    'Sodium_Intake',
    'HDL_Cholesterol',
    'Diastolic_Blood_Pressure',
    'Systolic_Blood_Pressure'
]

In [61]:
class ConvertToCategory(BaseEstimator):
    def __init__(self, columns):
        self.columns = columns
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()
        for col in self.columns:
            X[col] = X[col].astype('category')
        return X
    
class MissingCategoryAs999(BaseEstimator):
    def __init__(self, columns):
        self.columns = columns
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()
        for col in self.columns:
            X[col] = X[col].cat.add_categories(999).fillna(999)
        return X

In [77]:


param_dist_logistic = {
    'estimator': [LogisticRegression(random_state=SEED)],
    'estimator__C': uniform(0.1, 10),
    'estimator__penalty': ['l1', 'l2'],
    'estimator__solver': ['liblinear', 'saga']
}

param_dist_svc = {
    'estimator': [SVC(random_state=SEED)],
    'estimator__C': uniform(0.1, 10),
    'estimator__gamma': uniform(0.01, 1),
    'estimator__kernel': ['rbf', 'poly', 'sigmoid']
}

param_dist_rf = {
    'estimator': [RandomForestClassifier(random_state=SEED)],
    'estimator__n_estimators': randint(50, 200),
    'estimator__max_features': ['auto', 'sqrt', 'log2'],
    'estimator__max_depth': randint(1, 10),
    'estimator__criterion': ['gini', 'entropy']
}

param_dist_gb = {
    'estimator': [GradientBoostingClassifier(random_state=SEED)],
    'estimator__n_estimators': randint(50, 200),
    'estimator__learning_rate': uniform(0.01, 0.2),
    'estimator__max_depth': randint(1, 10)
}

# Common preprocessor parameters
preprocessor_params = {
    'preprocessor__numerical__imputer__strategy': ['mean', 'median']
}

# Combine all parameter distributions
param_dist = [
    #{**param_dist_logistic, **preprocessor_params},
    #{**param_dist_svc, **preprocessor_params},
    {**param_dist_rf, **preprocessor_params},
    #{**param_dist_gb, **preprocessor_params}
]

categorical_pipeline = Pipeline([
    ('convert_to_cat', ConvertToCategory(categorical_vars)),
    ('add_unknown_cat', MissingCategoryAs999(categorical_vars)),
])

numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer()),
    ('scaler', StandardScaler()),
])

preprocessor = ColumnTransformer([
    ('categorical', categorical_pipeline, categorical_vars),
    ('numerical', numerical_pipeline, numerical_vars)
])

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('estimator', LogisticRegression())  # This is a placeholder and will be replaced in RandomizedSearchCV
])

# Wrap the entire pipeline in a RandomizedSearchCV
final_pipeline = RandomizedSearchCV(
    pipeline,
    param_distributions=param_dist,
    n_iter=50,
    scoring='roc_auc',
    random_state=SEED,
    n_jobs=-1
)


In [78]:
final_pipeline.fit(X_train, y_train)

/root/miniconda3/envs/ds/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
80 fits failed out of a total of 250.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
39 fits failed with the following error:
Traceback (most recent call last):
  File "/root/miniconda3/envs/ds/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/root/miniconda3/envs/ds/lib/python3.12/site-packages/sklearn/base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/root/miniconda3/envs/ds/lib/python3.12/site-packages/sklearn/pipeline.py", l

RandomizedSearchCV(estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('categorical',
                                                                               Pipeline(steps=[('convert_to_cat',
                                                                                                ConvertToCategory(columns=['Race_ethnicity',
                                                                                                                           'General_health',
                                                                                                                           'Health_status',
                                                                                                                           'Told_High_Cholesterol',
                                                                                                                           'Household_income',
                                                                                                                           'Relative_Had_Diabetes'])),
                                                                                               ('add_unknown_cat',
                                                                                                MissingCategoryAs999(columns=['Race_ethnic...
                                         'estimator__max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f6f71c92db0>,
                                         'estimator__max_features': ['auto',
                                                                     'sqrt',
                                                                     'log2'],
                                         'estimator__n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f6eab288560>,
                                         'preprocessor__numerical__imputer__strategy': ['mean',
                                                                                        'median']}],
                   random_state=4208, scoring='roc_auc')

In [79]:
y_pred = final_pipeline.predict(X_test)
y_pred_proba = final_pipeline.predict_proba(X_test)[:, 1]

# Calculate metrics
auc = roc_auc_score(y_test, y_pred_proba)
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Best parameters found: ", final_pipeline.best_params_)
print("Best score found: ", final_pipeline.best_score_)
print("\nMetrics on test set:")
print(f"AUC: {auc:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1 Score: {f1:.4f}")


Best parameters found:  {'estimator': RandomForestClassifier(random_state=4208), 'estimator__criterion': 'entropy', 'estimator__max_depth': 9, 'estimator__max_features': 'sqrt', 'estimator__n_estimators': 178, 'preprocessor__numerical__imputer__strategy': 'median'}
Best score found:  0.8859594552295762

Metrics on test set:
AUC: 0.8942
Recall: 0.1335
Precision: 0.7887
F1 Score: 0.2284


### 3.3 Predictive Models

Models used in the paper:

- Logistic Regression
- Support Vector Machine
- Random Forest
- Gradient Boosted Trees
- Ensemble model of the 5 models.  

## 4. Model Comparison

In [ ]:
#jupyter nbconvert --to markdown R_replicate_Dinh.ipynb --output README.md